# LangGraph and LangSmith - Agentic RAG Powered by LangChain

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating our Tool Belt
  4. Creating Our State
  5. Creating and Compiling A Graph!

- 🤝 Breakout Room #2:
  1. Evaluating the LangGraph Application with LangSmith
  2. Adding Helpfulness Check and "Loop" Limits
  3. LangGraph for the "Patterns" of GenAI

# 🤝 Breakout Room #1

## Part 1: LangGraph - Building Cyclic Applications with LangChain

LangGraph is a tool that leverages LangChain Expression Language to build coordinated multi-actor and stateful applications that includes cyclic behaviour.

### Why Cycles?

In essence, we can think of a cycle in our graph as a more robust and customizable loop. It allows us to keep our application agent-forward while still giving the powerful functionality of traditional loops.

Due to the inclusion of cycles over loops, we can also compose rather complex flows through our graph in a much more readable and natural fashion. Effectively allowing us to recreate application flowcharts in code in an almost 1-to-1 fashion.

### Why LangGraph?

Beyond the agent-forward approach - we can easily compose and combine traditional "DAG" (directed acyclic graph) chains with powerful cyclic behaviour due to the tight integration with LCEL. This means it's a natural extension to LangChain's core offerings!

## Task 1:  Dependencies


## Task 2: Environment Variables

We'll want to set both our OpenAI API key and our LangSmith environment variables.

In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [2]:
os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY")

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE7 - LangGraph - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

## Task 3: Creating our Tool Belt

As is usually the case, we'll want to equip our agent with a toolbelt to help answer questions and add external knowledge.

There's a tonne of tools in the [LangChain Community Repo](https://github.com/langchain-ai/langchain-community/tree/main/libs/community) but we'll stick to a couple just so we can observe the cyclic nature of LangGraph in action!

We'll leverage:

- [Tavily Search Results](https://github.com/langchain-ai/langchain-community/blob/main/libs/community/langchain_community/tools/tavily_search/tool.py)
- [Arxiv](https://github.com/langchain-ai/langchain-community/blob/main/libs/community/langchain_community/tools/arxiv/tool.py)

#### 🏗️ Activity #1:

Please add the tools to use into our toolbelt.

> NOTE: Each tool in our toolbelt should be a method.

In [5]:
from langchain_tavily import TavilySearch
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

tavily_tool = TavilySearch(max_results=5)
wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

tool_belt = [
    tavily_tool,
    ArxivQueryRun(),
    wikipedia_tool
]

### Model

Now we can set-up our model! We'll leverage the familiar OpenAI model suite for this example - but it's not *necessary* to use with LangGraph. LangGraph supports all models - though you might not find success with smaller models - as such, they recommend you stick with:

- OpenAI's GPT-3.5 and GPT-4
- Anthropic's Claude
- Google's Gemini

> NOTE: Because we're leveraging the OpenAI function calling API - we'll need to use OpenAI *for this specific example* (or any other service that exposes an OpenAI-style function calling API.

In [6]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

Now that we have our model set-up, let's "put on the tool belt", which is to say: We'll bind our LangChain formatted tools to the model in an OpenAI function calling format.

In [7]:
model = model.bind_tools(tool_belt)

#### ❓ Question #1:

How does the model determine which tool to use?

#### Answer:

The model determines which tool to use through a thinking process much like us humans, logically deciding on which tool serves the purpose.

1. Tool Binding: We first equip the model with tools. Now the model can decide to use a particular tools as it sees fit.
2. Function calling: When the model receives a query, it analyzes what needs to be done and looks at its available tools.
3. Auto select: The model automatically selects the appropriate tool based on:
    - Nature of the query
    - The tools description and capabilities
    - the specific information that is required

## Task 4: Putting the State in Stateful

Earlier we used this phrasing:

`coordinated multi-actor and stateful applications`

So what does that "stateful" mean?

To put it simply - we want to have some kind of object which we can pass around our application that holds information about what the current situation (state) is. Since our system will be constructed of many parts moving in a coordinated fashion - we want to be able to ensure we have some commonly understood idea of that state.

LangGraph leverages a `StatefulGraph` which uses an `AgentState` object to pass information between the various nodes of the graph.

There are more options than what we'll see below - but this `AgentState` object is one that is stored in a `TypedDict` with the key `messages` and the value is a `Sequence` of `BaseMessages` that will be appended to whenever the state changes.

Let's think about a simple example to help understand exactly what this means (we'll simplify a great deal to try and clearly communicate what state is doing):

1. We initialize our state object:
  - `{"messages" : []}`
2. Our user submits a query to our application.
  - New State: `HumanMessage(#1)`
  - `{"messages" : [HumanMessage(#1)}`
3. We pass our state object to an Agent node which is able to read the current state. It will use the last `HumanMessage` as input. It gets some kind of output which it will add to the state.
  - New State: `AgentMessage(#1, additional_kwargs {"function_call" : "WebSearchTool"})`
  - `{"messages" : [HumanMessage(#1), AgentMessage(#1, ...)]}`
4. We pass our state object to a "conditional node" (more on this later) which reads the last state to determine if we need to use a tool - which it can determine properly because of our provided object!

In [8]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

## Task 5: It's Graphing Time!

Now that we have state, and we have tools, and we have an LLM - we can finally start making our graph!

Let's take a second to refresh ourselves about what a graph is in this context.

Graphs, also called networks in some circles, are a collection of connected objects.

The objects in question are typically called nodes, or vertices, and the connections are called edges.

Let's look at a simple graph.

![image](https://i.imgur.com/2NFLnIc.png)

Here, we're using the coloured circles to represent the nodes and the yellow lines to represent the edges. In this case, we're looking at a fully connected graph - where each node is connected by an edge to each other node.

If we were to think about nodes in the context of LangGraph - we would think of a function, or an LCEL runnable.

If we were to think about edges in the context of LangGraph - we might think of them as "paths to take" or "where to pass our state object next".

Let's create some nodes and expand on our diagram.

> NOTE: Due to the tight integration with LCEL - we can comfortably create our nodes in an async fashion!

In [9]:
from langgraph.prebuilt import ToolNode

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

tool_node = ToolNode(tool_belt)

Now we have two total nodes. We have:

- `call_model` is a node that will...well...call the model
- `tool_node` is a node which can call a tool

Let's start adding nodes! We'll update our diagram along the way to keep track of what this looks like!


In [10]:
from langgraph.graph import StateGraph, END

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)

Let's look at what we have so far:

![image](https://i.imgur.com/md7inqG.png)

Next, we'll add our entrypoint. All our entrypoint does is indicate which node is called first.

In [11]:
uncompiled_graph.set_entry_point("agent")

![image](https://i.imgur.com/wNixpJe.png)

Now we want to build a "conditional edge" which will use the output state of a node to determine which path to follow.

We can help conceptualize this by thinking of our conditional edge as a conditional in a flowchart!

Notice how our function simply checks if there is a "function_call" kwarg present.

Then we create an edge where the origin node is our agent node and our destination node is *either* the action node or the END (finish the graph).

It's important to highlight that the dictionary passed in as the third parameter (the mapping) should be created with the possible outputs of our conditional function in mind. In this case `should_continue` outputs either `"end"` or `"continue"` which are subsequently mapped to the action node or the END node.

In [12]:
def should_continue(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
    "agent",
    should_continue
)

Let's visualize what this looks like.

![image](https://i.imgur.com/8ZNwKI5.png)

Finally, we can add our last edge which will connect our action node to our agent node. This is because we *always* want our action node (which is used to call our tools) to return its output to our agent!

In [13]:
uncompiled_graph.add_edge("action", "agent")

Let's look at the final visualization.

![image](https://i.imgur.com/NWO7usO.png)

All that's left to do now is to compile our workflow - and we're off!

In [14]:
simple_agent_graph = uncompiled_graph.compile()

#### ❓ Question #2:

Is there any specific limit to how many times we can cycle?

If not, how could we impose a limit to the number of cycles?

#### Answer:

No, there is no inherent functionality that limits how many times the agents can cycle through the graph. We could however impose a limit to the number of cycles, by doing the following;

1. Counter method: Add a step counter that increments every time the graph goes through a cycle. Eg: Add a counter to the "AgentState" class, and increment it in the "should_continue" function
2. Time-based limit: We can also add a timestamp to the "AgentState", check duration in "should_continue" function and stop it if it exceeds a time limit.
3. Conditional check: Add completion criteria to "AgentState" and check condition in "should_continue" function

## Using Our Graph

Now that we've created and compiled our graph - we can call it *just as we'd call any other* `Runnable`!

Let's try out a few examples to see how it fairs:

In [15]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="Who is the current captain of the Winnipeg Jets?")]}

async for chunk in simple_agent_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_TMtrQu8HtjPaozH3xFvsN2tf', 'function': {'arguments': '{"query":"current captain of the Winnipeg Jets"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 931, 'total_tokens': 952, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': None, 'id': 'chatcmpl-BsFIidcSRk2LJOQ4lBURXnnt6pKsX', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--799a01df-94ed-4a5d-a014-929d383d68cc-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'current captain of the Winnipeg Jets'}, 'id': 'call_TMtrQu8HtjPaozH3xFvsN2tf', 'type': 'tool_call'}], usage_metad

Let's look at what happened:

1. Our state object was populated with our request
2. The state object was passed into our entry point (agent node) and the agent node added an `AIMessage` to the state object and passed it along the conditional edge
3. The conditional edge received the state object, found the "tool_calls" `additional_kwarg`, and sent the state object to the action node
4. The action node added the response from the OpenAI function calling endpoint to the state object and passed it along the edge to the agent node
5. The agent node added a response to the state object and passed it along the conditional edge
6. The conditional edge received the state object, could not find the "tool_calls" `additional_kwarg` and passed the state object to END where we see it output in the cell above!

Now let's look at an example that shows a multiple tool usage - all with the same flow!

In [16]:
inputs = {"messages" : [HumanMessage(content="Search Arxiv for the QLoRA paper, then search each of the authors to find out their latest Tweet using Tavily!")]}

async for chunk in simple_agent_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        if node == "action":
          print(f"Tool Used: {values['messages'][0].name}")
        print(values["messages"])

        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ipDF5SQumooDHa9wJ7SVrY3Z', 'function': {'arguments': '{"query": "QLoRA"}', 'name': 'arxiv'}, 'type': 'function'}, {'id': 'call_qYQcLZdqCBEhLxGJezCiNbfp', 'function': {'arguments': '{"query": "latest tweet by author"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 947, 'total_tokens': 998, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': None, 'id': 'chatcmpl-BsFOlydRwJh8USUk0W471P5YXZLcH', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--2969deca-1932-4add-aa35-53e017d6e513-0', tool_calls=[{'name': 'arxiv', 'args': {'query': 'QLoRA'}

#### 🏗️ Activity #2:

Please write out the steps the agent took to arrive at the correct answer.

#### Answer:

The agent's journey to answer the question about the QLoRA paper and its authors' tweets was very nteresting. Here is what happened:

1. The agent looked at the task and realized it needed two tools to process the request. Rather than doing these one at a time, it decided to request both simultaneously.
2. The Arxiv search was susccessful and found the original QLoRA paper from May 2023. It was able to extract author details from the paper. However, the Tavily search for tweets from the paper's authors failed and only returned generic tweets related to author content.
3. The agent recognized that it has a mix bag of results. Instead of failing silently, it shared the successful paper information, acknowledge the tweet search was not helpful, and offered to try a different approach for finding the tweets.

# 🤝 Breakout Room #2

## Part 1: LangSmith Evaluator

### Pre-processing for LangSmith

To do a little bit more preprocessing, let's wrap our LangGraph agent in a simple chain.

In [17]:
def convert_inputs(input_object):
  return {"messages" : [HumanMessage(content=input_object["question"])]}

def parse_output(input_state):
  return input_state["messages"][-1].content

agent_chain_with_formatting = convert_inputs | simple_agent_graph | parse_output

In [19]:
agent_chain_with_formatting.invoke({"question" : "What is RAG in machine learning??"})

"RAG in machine learning stands for Retrieval-Augmented Generation. It is a technique that combines large language models (LLMs) with retrieval systems, typically using a vector database, to improve the accuracy and relevance of responses, especially in domain-specific applications. \n\nThe process involves retrieving relevant documents or information from a vector database based on a user's query, and then incorporating this retrieved information into the context of the language model to generate a more informed and accurate response. This approach leverages the strengths of both retrieval systems and generative models, enabling more precise and context-aware outputs."

#### NOTE:

I changed the question to add some relevant context and help the agent return a relevant result. When I ran the code block as is, the agent searched teh wikipedia and returned "RAG" interms of "Durag". Interesting nuance. 

##### Other interesting ideas to control output
1. Query context analysis: The agent should analyze the query for specific domains or contexts. For rg: Academic/Research -> Arxiv
2. Tool descriptions can help the model decide which tool works best
3. Context specification in the input to handle ambiguous acronyms
4. Multi-step resolution process to validate context

### Task 1: Creating An Evaluation Dataset

Just as we saw last week, we'll want to create a dataset to test our Agent's ability to answer questions.

In order to do this - we'll want to provide some questions and some answers. Let's look at how we can create such a dataset below.

```python
questions = [
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "Who authored the QLoRA paper?",
    "What is the most popular deep learning framework?",
    "What significant improvements does the LoRA system make?"
]

answers = [
    {"must_mention" : ["paged", "optimizer"]},
    {"must_mention" : ["NF4", "NormalFloat"]},
    {"must_mention" : ["ground", "context"]},
    {"must_mention" : ["Tim", "Dettmers"]},
    {"must_mention" : ["PyTorch", "TensorFlow"]},
    {"must_mention" : ["reduce", "parameters"]},
]
```

#### 🏗️ Activity #3:

Please create a dataset in the above format with at least 5 questions.

In [20]:
questions = [
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "What are the three main innovations in QLoRA?",
    "What is the performance level compared to ChatGPT?",
    "What size GPU is needed for QLoRA finetuning?"
]

answers = [
    {"must_mention" : ["paged", "optimizer"]},
    {"must_mention" : ["NF4", "NormalFloat"]},
    {"must_mention" : ["ground", "context"]},
    {"must_mention": ["NF4", "double quantization", "paged optimizers"]},
    {"must_mention": ["99.3%", "ChatGPT"]},
    {"must_mention": ["48GB", "single"]}
]

Now we can add our dataset to our LangSmith project using the following code which we saw last Thursday!

In [21]:
from langsmith import Client

client = Client()

dataset_name = f"Retrieval Augmented Generation - Evaluation Dataset - {uuid4().hex[0:8]}"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions about the QLoRA Paper to Evaluate RAG over the same paper."
)

client.create_examples(
    inputs=[{"question" : q} for q in questions],
    outputs=answers,
    dataset_id=dataset.id,
)

{'example_ids': ['ec17f8ed-066c-445f-8920-73deffc01473',
  'd4359a8a-0f49-45cb-8e1b-0031bace091f',
  '46eaefbe-9cc6-444f-ad2c-552531e5bbf9',
  'dc0dd23b-4900-4a2f-888d-67f21ab32739',
  '51b1d04b-beb7-4fb1-89ae-87ce4e230212',
  '5daeca2e-d42f-4bbe-9221-3905c9a0af17'],
 'count': 6}

#### ❓ Question #3:

How are the correct answers associated with the questions?

> NOTE: Feel free to indicate if this is problematic or not

#### Answer:
The association between questions and answers happens ib the client.create_example() method through input-output matching. The output shows 6 example IDs, each pair is stored wit a unique ID.

It is problematic because;

1. The association relies purely on list position
2. No explicit linking between questions and their answers
3. if lists get out of syn, wrong answers could be associated
4. No semantic connection, just index-based matching

### Task 2: Adding Evaluators

Now we can add a custom evaluator to see if our responses contain the expected information.

We'll be using a fairly naive exact-match process to determine if our response contains specific strings.

In [23]:
from langsmith.evaluation import EvaluationResult, run_evaluator

@run_evaluator
def must_mention(run, example) -> EvaluationResult:
    prediction = run.outputs.get("output") or ""
    required = example.outputs.get("must_mention") or []
    score = all(phrase in prediction for phrase in required)
    return EvaluationResult(key="must_mention", score=score)

#### ❓ Question #4:

What are some ways you could improve this metric as-is?

> NOTE: Alternatively you can suggest where gaps exist in this method.

#### Answer:

##### Ways to improve the metric:

1. Semantic matching: The metric currently is only exact string matching. We can add semantic similarity checking, for eg: "NF4" should match "NormalFloat4" or "Normal Float 4"
2. Add case-insensitivity option
3. Proportional scoring instead of 1 or 0 scoring. For eg: If 2 out of 3 terms are present, score = 0.67

##### Gaps in current method:

1. Doesn't verify if concepts are presented in logical order. It cannot evaluate procedural correctness
2. It cannot currently detect if term is used in negative context.
3. Does not verufy relationships between terms
4. No way to indicate "close" answers

#### Task 3: Evaluating

All that is left to do is evaluate our agent's response!

In [24]:
experiment_results = client.evaluate(
    agent_chain_with_formatting,
    data=dataset_name,
    evaluators=[must_mention],
    experiment_prefix=f"Search Pipeline - Evaluation - {uuid4().hex[0:4]}",
    metadata={"version": "1.0.0"},
)

View the evaluation results for experiment: 'Search Pipeline - Evaluation - 945f-cbaa5738' at:
https://smith.langchain.com/o/f402e50c-d3db-4ba6-a176-44d754cac8d8/datasets/54b514cc-67ec-475b-8b64-03bea3176c26/compare?selectedSessions=db5343e8-9e08-4e56-86f5-b20c2dfb906e




0it [00:00, ?it/s]

In [25]:
experiment_results

<ExperimentResults Search Pipeline - Evaluation - 945f-cbaa5738>

## Part 2: LangGraph with Helpfulness:

### Task 3: Adding Helpfulness Check and "Loop" Limits

Now that we've done evaluation - let's see if we can add an extra step where we review the content we've generated to confirm if it fully answers the user's query!

We're going to make a few key adjustments to account for this:

1. We're going to add an artificial limit on how many "loops" the agent can go through - this will help us to avoid the potential situation where we never exit the loop.
2. We'll add to our existing conditional edge to obtain the behaviour we desire.

First, let's define our state again - we can check the length of the state object, so we don't need additional state for this.

In [26]:
class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

Now we can set our graph up! This process will be almost entirely the same - with the inclusion of one additional node/conditional edge!

#### 🏗️ Activity #5:

Please write markdown for the following cells to explain what each is doing.

##### Here we create a state graph with two main nodes:
1. "agent" node handles model calls
2. "action" node executes tool operations

In [28]:
graph_with_helpfulness_check = StateGraph(AgentState)

graph_with_helpfulness_check.add_node("agent", call_model)
graph_with_helpfulness_check.add_node("action", tool_node)

##### Let's set the "agent" node as the entry point to the loop

In [29]:
graph_with_helpfulness_check.set_entry_point("agent")

##### Lets implement a decision-making function that evaluates whether the agent's response is helpful enough to end the conversation or if it should continue processing, with a built-in limit to prevent infinite loops.

1. Tool call check, determines if the last message requires a tool action
2. Message length limit, prevents infinite loops by checking message count
3. Helpfulness evalution, uses "gpt-4.1-mini" to assess response quality
4. Decision making function returns "action", "end", or "continue" based on conditions

In [30]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def tool_call_or_helpful(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  initial_query = state["messages"][0]
  final_response = state["messages"][-1]

  if len(state["messages"]) > 10:
    return "END"

  prompt_template = """\
  Given an initial query and a final response, determine if the final response is extremely helpful or not. Please indicate helpfulness with a 'Y' and unhelpfulness as an 'N'.

  Initial Query:
  {initial_query}

  Final Response:
  {final_response}"""

  helpfullness_prompt_template = PromptTemplate.from_template(prompt_template)

  helpfulness_check_model = ChatOpenAI(model="gpt-4.1-mini")

  helpfulness_chain = helpfullness_prompt_template | helpfulness_check_model | StrOutputParser()

  helpfulness_response = helpfulness_chain.invoke({"initial_query" : initial_query.content, "final_response" : final_response.content})

  if "Y" in helpfulness_response:
    return "end"
  else:
    return "continue"

#### 🏗️ Activity #4:

Please write what is happening in our `tool_call_or_helpful` function!

1. The function is acting like a traffic controller for the agents conversation. It makes decisions about what should happen next based on the current state
2. First, checks if we need any tools. Then, check if there are more than 10 messages. Finally evaluates if the response is helpful or not
3. If the last message indicates we need to use a tool, immediately returns "action" to trigger tool use
4. Uses GPT-4 mini to compare original question and final answer and rates helpfulness with "Y" or "N"
5. Returns "end", if the response was helpful. Returns "continue" if we need to try again.


##### Here we setup decision pathways in our LangGraph, telling the agent where to go next based on the output of our tool_call_or_helpful fucntion

In [31]:
graph_with_helpfulness_check.add_conditional_edges(
    "agent",
    tool_call_or_helpful,
    {
        "continue" : "agent",
        "action" : "action",
        "end" : END
    }
)

##### Setting a return path to the "agent" node

In [32]:
graph_with_helpfulness_check.add_edge("action", "agent")

##### Agent graph compilation

In [33]:
agent_with_helpfulness_check = graph_with_helpfulness_check.compile()

##### Streaming agent responses with Async processing

In [34]:
inputs = {"messages" : [HumanMessage(content="Related to machine learning, what is LoRA? Also, who is Tim Dettmers? Also, what is Attention?")]}

async for chunk in agent_with_helpfulness_check.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_CmlhQP4iQCaIsoGp3ypAFNxi', 'function': {'arguments': '{"query": "LoRA machine learning"}', 'name': 'wikipedia'}, 'type': 'function'}, {'id': 'call_jmxFrGdOvbVftwe3LXrdEr0k', 'function': {'arguments': '{"query": "Tim Dettmers"}', 'name': 'wikipedia'}, 'type': 'function'}, {'id': 'call_ZnNdyFABKsYr1SgD3CqQ8Nm1', 'function': {'arguments': '{"query": "Attention in machine learning"}', 'name': 'wikipedia'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 946, 'total_tokens': 1010, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': None, 'id': 'chatcmpl-BsHhKguIurVI4rHw0DGOISL5jIiOI', 'service_tier': 'defa

/Users/ashapondicherry/Desktop/VDU/_Projects/AIE_Bootcamp/AIE_Bootcamp_VC/activate/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/ashapondicherry/Desktop/VDU/_Projects/AIE_Bootcamp/AIE_Bootcamp_VC/activate/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Receiving update from node: 'action'
[ToolMessage(content='Page: Fine-tuning (deep learning)\nSummary: In deep learning, fine-tuning is an approach to transfer learning in which the parameters of a pre-trained neural network model are trained on new data. Fine-tuning can be done on the entire neural network, or on only a subset of its layers, in which case the layers that are not being fine-tuned are "frozen" (i.e., not changed during backpropagation). A model may also be augmented with "adapters" that consist of far fewer parameters than the original model, and fine-tuned in a parameter-efficient way by tuning the weights of the adapters and leaving the rest of the model\'s weights frozen.\nFor some architectures, such as convolutional neural networks, it is common to keep the earlier layers (those closest to the input layer) frozen, as they capture lower-level features, while later layers often discern high-level features that can be more related to the task that the model is trained

### Task 4: LangGraph for the "Patterns" of GenAI

Let's ask our system about the 4 patterns of Generative AI:

1. Prompt Engineering
2. RAG
3. Fine-tuning
4. Agents

In [35]:
patterns = ["prompt engineering", "RAG", "fine-tuning", "LLM-based agents"]

In [36]:
for pattern in patterns:
  what_is_string = f"What is {pattern} and when did it break onto the scene??"
  inputs = {"messages" : [HumanMessage(content=what_is_string)]}
  messages = agent_with_helpfulness_check.invoke(inputs)
  print(messages["messages"][-1].content)
  print("\n\n")

Prompt engineering is the process of designing and refining prompts to effectively communicate with AI language models, such as GPT, to obtain desired responses. It involves crafting clear, specific, and contextually appropriate prompts to guide the AI's output, often requiring an understanding of how the model interprets input and generates responses.

Prompt engineering has gained prominence with the rise of large language models (LLMs) like GPT-3, which became widely accessible around 2020-2021. As these models demonstrated their ability to perform a wide range of tasks based on input prompts, the importance of prompt engineering as a skill and discipline also emerged. It became especially prominent in 2022 and 2023, as users and developers sought more effective ways to leverage LLMs for applications in writing, coding, customer service, and more.

In summary, prompt engineering is a crucial aspect of working with AI language models, and it started gaining significant attention arou

/Users/ashapondicherry/Desktop/VDU/_Projects/AIE_Bootcamp/AIE_Bootcamp_VC/activate/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/ashapondicherry/Desktop/VDU/_Projects/AIE_Bootcamp/AIE_Bootcamp_VC/activate/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Retrieval-Augmented Generation (RAG) is a technique that enhances large language models (LLMs) by enabling them to retrieve and incorporate new information from external sources before generating responses. This approach allows LLMs to access domain-specific or updated information that is not part of their original training data, improving accuracy and relevance. 

RAG was first introduced in a research paper from Meta in 2020. It helps reduce hallucinations in AI responses, allows for the inclusion of sources for verification, and can be more cost-effective by reducing the need for retraining models with new data.



Fine-tuning is a machine learning process where a pre-trained model is further trained on a specific dataset to adapt it to a particular task or domain. This approach leverages the knowledge the model has already gained during its initial training, allowing it to perform well on specialized tasks with less additional data and training time.

Fine-tuning became prominent w